<a href="https://colab.research.google.com/github/chentong1023/bert/blob/master/Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 关联推理-文本分类大作业（MNLI）

姓名：陈彤

学号：518030910416

模型选择：bert(BERT-BASE)+keras

加载数据

In [0]:
import numpy as np
import csv
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense
from keras.utils.np_utils import to_categorical

set_gelu('tanh')

num_classes = 3
maxlen = 128
batch_size = 32
config_path = './bert_config.json'
checkpoint_path = './bert_model.ckpt'
dict_path = './vocab.txt'


def get_Ids(first_text, second_text):
  fir = first_text.lower()
  sec = second_text.lower()
  return tokenizer.encode(first_text=fir,second_text=sec)

def load_train(filename):
    D = []
    def trans_label(label):
      if (label == 'entailment'):
        return 0
      elif (label == 'neutral'):
        return 1
      else:
        return 2
    with open(filename, encoding='utf-8') as f:
      rows = csv.DictReader(f)
      for row in rows:
        D.append(([str(row['premise1']) + str(row['premise2']) + str(row['premise3']) + str(row['premise4']), str(row['hypothesis'])], to_categorical(trans_label(row['label']), num_classes=num_classes) ))
    return D

def load_test(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
      rows = csv.DictReader(f)
      for row in rows:
        D.append([str(row['premise1']) + str(row['premise2']) + str(row['premise3']) + str(row['premise4']), str(row['hypothesis'])])
    return D

# 加载数据集
train_data = load_train('./train.csv')
test_data = load_test('./test.csv')

train_token = []
test_token = []

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

train_str = []
train_label = []
for text, label in train_data:
  token_ids, segment_ids = tokenizer.encode(text[0], text[1], max_length=maxlen)
  train_token.append(([token_ids, segment_ids], label))

for text in test_data:
  token_ids, segment_ids = tokenizer.encode(text[0], text[1], max_length=maxlen)
  test_token.append([token_ids, segment_ids])

class data_generator(DataGenerator):
    # 数据生成器
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = text[0], text[1]
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(label)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], [batch_labels]
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

预训练

In [0]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='bert',
    return_keras_model=False,
)

output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
AdamLR = extend_with_piecewise_linear_lr(Adam, name='Adam')

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(5e-8),  # 用足够小的学习率
    
    #optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
    #    1000: 1,
    #    2000: 0.1
    #}),
    metrics=['categorical_accuracy'],
)

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
___________________________________________________________________________________________

训练和评估

In [0]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        token_ids, segment_ids = get_Ids(x_test[0],x_test[1])
        y_pred = model.predict([np.array([token_ids]),np.array([segment_ids])]).argmax(axis=1)
        y_indx = np.array(y_true).argmax()
        total += 1
        right += (y_indx == y_pred)
    return right / total


class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
      val_acc = evaluate(valid_generator)
      if val_acc > self.best_val_acc:
          self.best_val_acc = val_acc
          model.save_weights('best_model.weights')
      print(
          u'val_acc: %.5f, best_val_acc: %.5f\n' %
          (val_acc, self.best_val_acc)
      )

train_generator = data_generator(train_token, batch_size)
valid_generator = data_generator(train_token, batch_size)
evaluator = Evaluator()
model.load_weights("best_model.weights")
model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=10
    ,callbacks=[evaluator]
)
model.load_weights("best_model.weights")

Epoch 1/10
282/282 [==============================] - 336s 1s/step - loss: 0.1745 - categorical_accuracy: 0.9447
val_acc: 0.37943, best_val_acc: 0.37943

Epoch 2/10
282/282 [==============================] - 337s 1s/step - loss: 0.1685 - categorical_accuracy: 0.9474
val_acc: 0.37943, best_val_acc: 0.37943

Epoch 3/10
282/282 [==============================] - 336s 1s/step - loss: 0.1701 - categorical_accuracy: 0.9460
val_acc: 0.37943, best_val_acc: 0.37943

Epoch 4/10
282/282 [==============================] - 337s 1s/step - loss: 0.1674 - categorical_accuracy: 0.9473
val_acc: 0.37943, best_val_acc: 0.37943

Epoch 5/10
282/282 [==============================] - 336s 1s/step - loss: 0.1629 - categorical_accuracy: 0.9474
val_acc: 0.37943, best_val_acc: 0.37943

Epoch 6/10
282/282 [==============================] - 335s 1s/step - loss: 0.1605 - categorical_accuracy: 0.9490
val_acc: 0.37943, best_val_acc: 0.37943

Epoch 7/10
282/282 [==============================] - 336s 1s/step - loss: 0

输出预测

In [0]:
# 转换数据集

label_names = ["entailment","neutral","contradiction"]
pred_label=[]
cnter = 0
for x_test in test_data:
  if cnter <= 20:
    print (x_test)
  token_ids, segment_ids = get_Ids(x_test[0],x_test[1])
  y_pred = model.predict([np.array([token_ids]),np.array([segment_ids])]).argmax(axis=1)
  pred_label.append(label_names[y_pred[0]])
  if cnter <= 20:
    print(y_pred)
    cnter = cnter+1
with open("predict.csv", "w") as fo:
  writer = csv.writer(fo)
  i=0
  writer.writerow(["ID", "label"])
  for lb in pred_label:
    writer.writerow([i, lb])
    i=i+1

['Three male safety workers in neon yellow vests are taking their break.Three workers in bright green vests, taking a break.Three construction workers sit down and take a break.Construction workers have a hard job.', 'Men sit on a bench.']
[1]
['Several young adults sit in a room with a projector on the ceiling.Students sitting in a classroom are learning about art.A bunch of kids in a classroom not doing any work.A meeting in a room with six people.', 'A group sitting around a table.']
[0]
['Young women and men on a stairwell talking with pictures of the United States behind them.Woman in tie-dye shirt and another woman sing on the steps.Young adults sitting on the stairs talking.Young people sitting on steps.', 'A group sits.']
[0]
['Children stand with guns on a large green lawn with a man dressed as a soldier.A man dressed as a British Guard entertains children holding toy guns.Kids are watching a man in uniform standing on the grass.A soldier is teaching kids how to handle a gun.'

以下为环境的配置：

In [0]:
!pip install git+https://www.github.com/bojone/bert4keras.git

  Cloning https://www.github.com/bojone/bert4keras.git to /tmp/pip-req-build-boyio5_a
  Running command git clone -q https://www.github.com/bojone/bert4keras.git /tmp/pip-req-build-boyio5_a
  Created wheel for bert4keras: filename=bert4keras-0.7.7-cp36-none-any.whl size=40962 sha256=9b8eb88f9007556d485bae87b1687d891f9e0c4efe5b3d734f2c0de3e7f315d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-f7i1z701/wheels/12/58/83/8ff5c864b80c860e6d9e9e0d90c04fafca05d01d21f9f6fcba
Successfully built bert4keras


In [0]:
!wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip

--2020-05-20 06:50:41--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 389.20M   133MB/s    in 2.9s    

2020-05-20 06:50:44 (133 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [408102251/408102251]



In [0]:
!pip install bert4keras

In [0]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_config.json        
  inflating: vocab.txt               
  inflating: bert_model.ckpt.index   
